# **Template OP on salish**

In [14]:
%matplotlib inline

import sys
import xarray as xr
import numpy as np
import os
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, AdvectionRK4_3D

sys.path.append('/home/jvalenti/MOAD/analysis-jose/notebooks/parcels')
from Kernels import DeleteParticle, Buoyancy
from OP_functions import *

In [15]:
# Define paths
local = 0 #Set to 0 when working on server
paths = path(local)

In [16]:
Dat=xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
path_NEMO = make_prefix(datetime(2018, 12, 23), paths['NEMO'])
Dat0=xr.open_dataset(path_NEMO + '_grid_W.nc')
Dat=xr.open_dataset(path_NEMO + '_grid_T.nc')
coord=xr.open_dataset(paths['mask'],decode_times=False)
WW3 = xr.open_dataset(get_WW3_path(datetime(2018, 12, 23)))
coord['nav_lat']

<xarray.DataArray 'nav_lat' (y: 898, x: 398)>
[357404 values with dtype=float32]
Dimensions without coordinates: y, x
Attributes:
    standard_name:  nav_lat
    long_name:      latitude of T-grid points
    units:          degrees_north

## Definitions

In [17]:
start = datetime(2019, 12, 10) #Start date
# Set Time length [days] and timestep [seconds]
length = 10
duration = timedelta(days=length)
dt = 90 #toggle between - or + to pick backwards or forwards
N = 6 # number of deploying locations
n = 100 # 1000   # number of particles per location
dmin = [0,0,0,0,0,70] #minimum depth
dd = 20 #max depth difference from dmin
x_offset, y_offset, zvals = p_deploy(N,n,dmin,dd)

In [18]:
from parcels import Variable

class PEParticle(JITParticle):         # Define a new particle class
    ro = Variable('ro', initial = 960)             # desnsity PE
    size = Variable('size', initial = 5e-5)

class BParticle(JITParticle):         # Define a new particle class
    beached = Variable('beached', initial = 0) #Keep track of beached particles

In [19]:
Dat=xr.open_dataset(paths['coords'],decode_times=False)
outf_lat=Dat['nav_lat'][445,304]
outf_lon=Dat['nav_lon'][445,304]

In [20]:
lon = np.zeros([N,n])
lat = np.zeros([N,n])
# Execute run
clon, clat = [-123.901172,-125.155849,-123.207648,-122.427508,-123.399769,float(outf_lon)], [49.186308,49.975326,49.305448,47.622403,48.399420,float(outf_lat)]  # choose horizontal centre of the particle cloud
for i in range(N):
    lon[i,:]=(clon[i] + x_offset[i,:])
    lat[i,:]=(clat[i] + y_offset[i,:])
z = zvals

In [21]:
#Set start date time and the name of the output file
name = 'Wavesss' #name output file
daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.nc'
outfile = os.path.join(paths['out'], fn)
print(outfile)

/home/jvalenti/MOAD/analysis-jose/notebooks/results/Wavesss20191210_1n_20191220_1n.nc


## Simulation

In [22]:
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W','R']
filenames,variables,dimensions=filename_set(start,length,varlist,local)
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

varlist=['US','VS','WL']
filenames,variables,dimensions=filename_set(start,length,varlist,local)

us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))


filenames,variables,dimensions=filename_set(start,length,['Bathy'],local)
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)

In [23]:
def Buoyancy(particle, fieldset, time):
    '''Stokes law settling velocity'''
    Rp = particle.ro #Density particle: LDPE (~920 kg/m3 ),PS (~150 kg/m3), PET (~1370 kg/m3).
    ESD= particle.size #Size particle (ESD) equivalent spherical diameter
    visc=1e-3 #average viscosity sea water
     
    z = particle.depth
    u = fieldset.U[time, particle.depth, particle.lat, particle.lon]
    if  u == 0:
        dz = 0
    else:
        g = 9.8
        #t = fieldset.T[time, particle.depth, particle.lat, particle.lon]
        ro = fieldset.R[time, particle.depth, particle.lat, particle.lon]
        #visc = 4.2844e-5 + 1/(0.157*((t + 64.993)**2)-91.296)
        Ws= ((ESD**2)*g*(Rp-ro))/(18*visc)
        WWS =(11.68 + 0.1991*ESD*1e6 + 0.0004*(ESD*1e6)**2- 0.0993*(Rp-ro) + 0.0002*(Rp-ro)**2)/86400
        if Ws > WWS:
            dz = WWS*particle.dt
        else:
            dz = Ws*particle.dt
    if dz+z > 0:
        particle.depth += dz 
    else:
        particle.depth = 0.5

In [25]:
def baty(particle, fieldset, time):
    bath = fieldset.mbathy[time, particle.depth, particle.lat, particle.lon]

In [26]:
# #Load Salish output as fields
#field_set = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
pset = ParticleSet.from_list(field_set, JITParticle, lon=lon, lat=lat, depth=z, time=start+timedelta(hours=2))

p_new = pset.Kernel(baty) #uncomment if you want to use additional kernels, also change the value of pset.Kernel to "AdvectionRK4_3D + p_flt"
##pset.computeTimeChunk(allow_time_extrapolation=1)
pset.execute(AdvectionRK4_3D + p_new, 
             runtime=duration, 
             dt=dt,
             output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=1)),
             recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayJITParticleAdvectionRK4_3Dbaty ==> /tmp/parcels-2894/lib85353da563a401ff3825ada3a42accfc_0.so
INFO: Temporary output files are stored in /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-WXAKTTPZ.
INFO: You can use "parcels_convert_npydir_to_netcdf /home/jvalenti/MOAD/analysis-jose/notebooks/results/out-WXAKTTPZ" to convert these to a NetCDF file during the run.
  8% (70200.0 of 864000.0) |#            | Elapsed Time: 0:00:31 ETA:   0:07:36WARNING: File /opp/wwatch3/nowcast/10dec19/SoG_ww3_fields_20191210_20191210.nc could not be decoded properly by xarray (version 0.19.0).
         It will be opened with no decoding. Filling values might be wrongly parsed.
 13% (118800.0 of 864000.0) |#           | Elapsed Time: 0:00:58 ETA:   0:07:24